In [2]:
import requests
import pandas as pd

# Get the local data
# Define the API endpoint
api_url = "http://127.0.0.1:5000/api/data"

# Call the API
try:
    response = requests.get(api_url, timeout=10)

    # Check if the response is successful
    if response.status_code == 200:
        data = response.json()
        bike_df = pd.DataFrame(data)
        print("Header of the table:")
        print(bike_df.head())
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")

Header of the table:
  Bike_model  Bike_number       End_date                  End_station  \
0    CLASSIC        23715  8/1/2023 0:17  Albert Embankment, Vauxhall   
1    CLASSIC        41267  8/1/2023 0:17  Albert Embankment, Vauxhall   
2    CLASSIC        53180  8/1/2023 0:11   Baldwin Street, St. Luke's   
3    CLASSIC        53431  8/1/2023 0:12      Grosvenor Road, Pimlico   
4    CLASSIC        54758  8/1/2023 0:17      Vauxhall Walk, Vauxhall   

   End_station_number   Latitude Longitude     Number     Start_date  \
0                1059  51.486343 -0.122492  132825189  8/1/2023 0:00   
1                1059  51.486343 -0.122492  132825190  8/1/2023 0:00   
2                3500  51.526236 -0.134407  132825191  8/1/2023 0:00   
3                1140  51.490945  -0.18119  132825192  8/1/2023 0:01   
4              200056  51.500139 -0.113936  132825193  8/1/2023 0:01   

                           Start_station  Start_station_number  \
0  Kennington Lane Rail Bridge, Vauxhall 

In [3]:
# https://api-portal.tfl.gov.uk/api-details#api=BikePoint&operation=BikePoint_GetAll

import requests
import pprint

from config import tfl_api_key

# Get the API data test
# Example API endpoint for getting bus stops near a location
# url = f"https://api.tfl.gov.uk/BikePoint/Search?query=Brunswick+Square+Bloomsbury"

# TfL API endpoint for searching bike points
url_template = "https://api.tfl.gov.uk/BikePoint/Search?query={}"
url = url_template.format('Brunswick Square, Bloomsbury')

# Set headers with your API key
headers = {"Authorization": f"AppKey {tfl_api_key}"}

response = requests.get(url, headers=headers)

# Check for successful response
if response.status_code == 200:
  data = response.json()
  # Process the data (list of bus stops)
  pprint.pprint(data)
  # Print target lat and lon data
  print(data[0]['lat'])
  print(data[0]['lon'])
else:
  print(f"Error: {response.status_code}")


[{'$type': 'Tfl.Api.Presentation.Entities.Place, Tfl.Api.Presentation.Entities',
  'additionalProperties': [],
  'children': [],
  'childrenUrls': [],
  'commonName': 'Brunswick Square, Bloomsbury',
  'id': 'BikePoints_11',
  'lat': 51.523951,
  'lon': -0.122502,
  'placeType': 'BikePoint',
  'url': '/Place/BikePoints_11'}]
51.523951
-0.122502


In [4]:
# Combine stations into a single list
all_stations = bike_df['Start_station'].tolist() + bike_df['End_station'].tolist()

# Remove duplicates and sort alphabetically
unique_stations = sorted(set(all_stations))

# Fix known issue with one station
station = 'Kennington Road  , Vauxhall'
index = unique_stations.index(station)
station_corrected = 'Kennington Road, Vauxhall'
unique_stations[index] = station_corrected
station_corrected
len(unique_stations)


799

In [5]:
import requests
from requests.adapters import HTTPAdapter, Retry
import pandas as pd

# Setup retry strategy
retries = Retry(
    total=3,  # Try up to 3 times
    backoff_factor=2,  # Double the wait time between retries
    status_forcelist=[429],  # Retry only on 429 errors
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)

# Create a session and mount the adapter with retry strategy
session = requests.Session()
adapter = HTTPAdapter(max_retries=retries)
session.mount('https://', adapter)

# TfL API endpoint for searching bike points
url_template = "https://api.tfl.gov.uk/BikePoint/Search?query={}"

# Function to call TfL API and parse response
def get_station_info(station_name):
    url = url_template.format(station_name)
    try:
        response = session.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx, 5xx)
        data = response.json()
        if data:
            # Get target data and return for DataFrame
            latitude = data[0].get('lat')
            longitude = data[0].get('lon')
            return pd.Series({'station_name': station_name, 'latitude': latitude, 'longitude': longitude})
        else:
            print(f"Location data not found for {station_name}")
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err} for station {station_name}")
    except Exception as err:
        print(f"An error occurred: {err} for station {station_name}")
    return None  # Indicate no data found

# Get unique station names
stations = list(set([*bike_df['Start_station'], *bike_df['End_station']]))

# Create empty DataFrame
df_stations = pd.DataFrame(columns=['station_name', 'latitude', 'longitude'])

# Call TfL API for each station and collect data
for station in unique_stations:
  station_info = get_station_info(station)
  if station_info is not None:
    df_stations = pd.concat([df_stations, station_info.to_frame().T],ignore_index=True)

# Print DataFrame
print(df_stations)

Location data not found for Chancery Lane, Holborn
Location data not found for Eaton Square, Belgravia
Location data not found for Limerston Street, West Chelsea
Location data not found for Mechanical Workshop Clapham
Location data not found for Mechanical Workshop Penton
Location data not found for Portman Square, Marylebone
Location data not found for Southwark Station 1, Southwark
Location data not found for Stonecutter Street, Holborn
Location data not found for Strand, Strand_OLD
Location data not found for Watney Market, Stepney
                          station_name   latitude longitude
0    Abbey Orchard Street, Westminster  51.498125 -0.132102
1        Abbotsbury Road, Holland Park  51.501391 -0.205991
2      Aberdeen Place, St. John's Wood  51.524826 -0.176268
3             Aberfeldy Street, Poplar  51.513548 -0.005659
4          Abingdon Green, Westminster   51.49764 -0.125972
..                                 ...        ...       ...
784               Wren Street, Holborn 

In [6]:
# Confirm fix for station
df_stations.loc[df_stations['station_name'] == 'Kennington Road, Vauxhall']


,station_name,latitude,longitude
381,"Kennington Road, Vauxhall",51.495718,-0.110889


In [7]:
# Create big .csv from main file

import zipfile
import os

def extract_csv_from_zip_and_save(zip_file_path, csv_file_name_inside_zip):
    try:
        temp_dir = 'temp'
        os.makedirs(temp_dir, exist_ok=True)

        # Open the ZIP file for reading
        with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
            # Check if the CSV file exists in the ZIP archive
            if csv_file_name_inside_zip in zip_file.namelist():
                # Extract the CSV file from the ZIP archive to a temporary location
                zip_file.extract(csv_file_name_inside_zip, temp_dir)

    except Exception as e:
        print(f"Error occurred: {e}")

    return os.path.join(temp_dir, csv_file_name_inside_zip)

# Example usage
zip_file_path = '../static/data/LondonBikeJourneyAug2023.csv.zip'
csv_file_name_inside_zip = 'LondonBikeJourneyAug2023.csv'
csv_file_path = extract_csv_from_zip_and_save(zip_file_path, csv_file_name_inside_zip)

In [8]:
import pandas as pd

# Load extracted CSV from the temp folder
temp_csv_file_path = os.path.join('temp', 'LondonBikeJourneyAug2023.csv')
temp_df = pd.read_csv(temp_csv_file_path)

temp_df.head()

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
0,132825189,8/1/2023 0:00,1190,"Kennington Lane Rail Bridge, Vauxhall",8/1/2023 0:17,1059,"Albert Embankment, Vauxhall",23715,CLASSIC,16m 46s,1006663
1,132825190,8/1/2023 0:00,1190,"Kennington Lane Rail Bridge, Vauxhall",8/1/2023 0:17,1059,"Albert Embankment, Vauxhall",41267,CLASSIC,16m 47s,1007128
2,132825191,8/1/2023 0:00,983,"Euston Road, Euston",8/1/2023 0:11,3500,"Baldwin Street, St. Luke's",53180,CLASSIC,11m 6s,666395
3,132825192,8/1/2023 0:01,3479,"Old Brompton Road, South Kensington",8/1/2023 0:12,1140,"Grosvenor Road, Pimlico",53431,CLASSIC,11m 53s,713059
4,132825193,8/1/2023 0:01,1219,"Lower Marsh, Waterloo",8/1/2023 0:17,200056,"Vauxhall Walk, Vauxhall",54758,CLASSIC,16m 3s,963786


In [10]:
# Fix one station address
station = 'Kennington Road  , Vauxhall'
station_corrected = 'Kennington Road, Vauxhall'

# Define replacement dictionary
replacements = {"Start station": {station: station_corrected},
                "End station": {station: station_corrected}}

# Apply replacements using vectorized operations
temp_df.update(temp_df.replace(replacements))


In [15]:
# Verify change
first_occurrence = temp_df[temp_df["Start station"] == station_corrected].index.min()

# Print the row (assuming the index is an integer)
print(temp_df.iloc[first_occurrence])


Number                                     132825198
Start date                             8/1/2023 0:01
Start station number                            1154
Start station              Kennington Road, Vauxhall
End date                               8/1/2023 0:09
End station number                              1093
End station             Kennington Cross, Kennington
Bike number                                    53424
Bike model                                   CLASSIC
Total duration                                7m 16s
Total duration (ms)                           436692
Name: 9, dtype: object


In [26]:
# Merge station coordinates onto the CSV
merged_df = pd.merge(temp_df, df_stations, left_on="Start station", right_on="station_name", how="left")

# Rename start columns
merged_df = merged_df.rename(columns={'latitude':'start_lat', 'longitude': 'start_lon'})

# Drop column to refresh for another merge
merged_df = merged_df.drop('station_name', axis=1)

# Merge again for end stations
merged_df = pd.merge(merged_df, df_stations, left_on="End station", right_on="station_name", how="left")

# Rename end columns
merged_df = merged_df.rename(columns={'latitude':'end_lat', 'longitude': 'end_lon'})

# Drop uneccessary column
merged_df = merged_df.drop('station_name', axis=1)

# Display the merged DataFrame
merged_df.head()

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms),start_lat,start_lon,end_lat,end_lon
0,132825189,8/1/2023 0:00,1190,"Kennington Lane Rail Bridge, Vauxhall",8/1/2023 0:17,1059,"Albert Embankment, Vauxhall",23715,CLASSIC,16m 46s,1006663,51.486343,-0.122492,51.490435,-0.122806
1,132825190,8/1/2023 0:00,1190,"Kennington Lane Rail Bridge, Vauxhall",8/1/2023 0:17,1059,"Albert Embankment, Vauxhall",41267,CLASSIC,16m 47s,1007128,51.486343,-0.122492,51.490435,-0.122806
2,132825191,8/1/2023 0:00,983,"Euston Road, Euston",8/1/2023 0:11,3500,"Baldwin Street, St. Luke's",53180,CLASSIC,11m 6s,666395,51.526236,-0.134407,51.527025,-0.088542
3,132825192,8/1/2023 0:01,3479,"Old Brompton Road, South Kensington",8/1/2023 0:12,1140,"Grosvenor Road, Pimlico",53431,CLASSIC,11m 53s,713059,51.490945,-0.18119,51.485357,-0.142207
4,132825193,8/1/2023 0:01,1219,"Lower Marsh, Waterloo",8/1/2023 0:17,200056,"Vauxhall Walk, Vauxhall",54758,CLASSIC,16m 3s,963786,51.500139,-0.113936,51.488124,-0.120903


# The zip wasn't well compressed so I zipped it manually and put it in the static/data folder

In [27]:
# Path to save the merged CSV file
merged_csv_file_path = 'merged_LondonBikeJourneyAug2023.csv'
merged_df.to_csv(merged_csv_file_path, index=False)

# Path for the output ZIP file
zip_file_path = 'UpdateLondonBikeJourneyAug2023.zip'

# Create a new ZIP file
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    # Add the CSV file to the ZIP archive
    zipf.write(merged_csv_file_path, os.path.basename(merged_csv_file_path))

print(f"CSV file '{merged_csv_file_path}' has been zipped to '{zip_file_path}'.")

CSV file 'merged_LondonBikeJourneyAug2023.csv' has been zipped to 'UpdateLondonBikeJourneyAug2023.zip'.
